In [1]:
import os
import re
import torch as T
import torch.nn.functional as F
import pandas as pd

In [2]:
import sys
sys.path.append("../scripts/")

In [3]:
from bert4rec_model import RecommendationModel
from bert4rec_dataset import Bert4RecDataset

In [4]:
data = pd.read_csv("../data/ratings_mapped.csv")

In [5]:
data.head()

,userId,movieId,rating,timestamp,movieId_mapped
0,429,595,5.0,828124615,514
1,429,588,5.0,828124615,508
2,429,590,5.0,828124615,510
3,429,592,5.0,828124615,511
4,429,432,3.0,828124615,378


In [6]:
rec_model = RecommendationModel(9725)

NUM POS:  120


In [7]:
from AttentionTransformer.utilities import count_model_parameters

In [8]:
# count_model_parameters(rec_model) / 1e6

In [9]:
dataset = Bert4RecDataset(data_csv=data,
                          group_by_col="userId",
                          data_col="movieId_mapped")


In [10]:
from torch.utils.data import DataLoader

In [11]:
dataloader = DataLoader(dataset, batch_size = 8, shuffle = True)

In [12]:
batch_1 = next(iter(dataloader))

In [13]:
batch_1.keys()

dict_keys(['source', 'target', 'source_mask', 'target_mask'])

In [14]:
model_op = rec_model(batch_1["source"])

In [15]:
model_op.size()

torch.Size([8, 120, 9725])

In [16]:
mask = batch_1["source"] == 1

In [18]:
batch_1["target"].size()

torch.Size([8, 120])

In [28]:
_, predicted = model_op.max(2)


In [29]:
predicted.size()

torch.Size([8, 120])

In [31]:
mask.size()

torch.Size([8, 120])

In [30]:
## accuracy
y_true = T.masked_select(batch_1["target"], mask)
predicted = T.masked_select(predicted, mask)

In [33]:
(y_true == predicted).double().mean()

tensor(0., dtype=torch.float64)

In [34]:
model_op.size()

torch.Size([8, 120, 9725])

In [35]:
model_op = model_op.view(-1, model_op.size(2))

In [36]:
model_op.size()

torch.Size([960, 9725])

In [37]:
batch_1["target"].view(-1).size()

torch.Size([960])

In [38]:
## loss
loss = F.cross_entropy(model_op, batch_1["target"].view(-1), reduction="none")
loss = loss * mask.view(-1)
loss.sum() / (mask.sum() + 1e-8)

tensor(9.3746, grad_fn=<DivBackward0>)

In [39]:
type(model_op), type(batch_1["target"]), type(mask)

(torch.Tensor, torch.Tensor, torch.Tensor)

In [40]:
type(loss)

torch.Tensor

In [43]:
(T.tensor([1, 2, 3]) == T.tensor([1, 0, 3])).double().mean()

tensor(0.6667, dtype=torch.float64)

In [44]:
type(rec_model)

bert4rec_model.RecommendationModel

In [45]:
type(dataloader)

torch.utils.data.dataloader.DataLoader

In [47]:
means = [1, 2, 3.3, 4.5, 6.6, 7, 8]
batch_sizes = [8, 8, 8, 8, 8, 8, 4]
accuracy = T.sum(T.tensor(batch_sizes) * T.tensor(means)) / T.sum(
        T.tensor(batch_sizes))

In [49]:
accuracy.item()

4.3692307472229